### Load dataset

In [48]:
import pandas as pd
RANDOM_STATE = 42

%run "DataHelpers.ipynb"

# Can be replaced with desired variant for different feature sets
GENE_FILE_VARIANT = FeatureVariant.RESEARCH_LOG # For values, see FeatureVariant.print_info()
variant = ModelVariant.LG                      # For values, see ModelVariant.print_info()

FILE_PATH = f"../Data/patient_genes_{GENE_FILE_VARIANT}.csv"

df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   FYN      977 non-null    float64
 1   BRCA1    977 non-null    float64
 2   FOXC1    977 non-null    float64
 3   TBC1D1   977 non-null    float64
 4   LAG3     977 non-null    float64
 5   CDK6     977 non-null    float64
 6   GATA3    977 non-null    float64
 7   CCND1    977 non-null    float64
 8   PRR4     977 non-null    float64
 9   EPCAM    977 non-null    float64
 10  CD274    977 non-null    float64
 11  PIK3CA   977 non-null    float64
 12  TOP2A    977 non-null    float64
 13  DCLK1    977 non-null    float64
 14  MYC      977 non-null    float64
 15  LRPPRC   977 non-null    float64
 16  BRCA2    977 non-null    float64
 17  TP53     977 non-null    float64
 18  EGFR     977 non-null    float64
 19  MKI67    977 non-null    float64
 20  TTN      977 non-null    float64
 21  CTLA4    977 non

### Import model function

In [49]:
from sklearn.linear_model import LogisticRegression

### Dataset split: training and test data

In [50]:
X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
print("\nApplied Smote")
X_smote, y_smote, X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote = split_data_apply_smote(df, "tnbc")

X_train.shape=(781, 31)
X_test.shape=(196, 31)
y_train.shape=(781,)
y_test.shape=(196,)

Applied Smote
X_train.shape=(1379, 31)
X_test.shape=(345, 31)
y_train.shape=(1379,)
y_test.shape=(345,)


### Logistic Regression

In [51]:
# Create model
# Bumping max_iter to a higer number than the default 100, MAY resolve the following warning
#       ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.
# TODO: Look into scaling the data

model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=100_000)


def run_model(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, test_case_ids, is_smote: bool):
    # Train the model
    model.fit(X_train, y_train)

    # Model predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

    # Save it in a dataframe, to CSV
    predictions = pd.DataFrame({
        "case_id": test_case_ids,
        "y_test": y_test,
        "y_pred": y_pred,
        "y_prob": y_prob
    })
    predictions.to_csv(f"../Data/model_output_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)

    return y_pred, y_prob

In [52]:
y_pred, y_prod = run_model(X_train, X_test, y_train, y_test, test_case_ids, False)

print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.95


## Smote applied

In [53]:
y_pred_smote, y_prod_smote = run_model(X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote, True)

print_evaluated_model_accuracy(y_test_smote, y_pred_smote)

Accuracy: 0.96


## Model cross validation

In [54]:
def run_cross_validation(X: pd.DataFrame, y: pd.Series, y_test: pd.Series, y_pred: pd.Series, y_prob: pd.Series, is_smote: bool) -> pd.DataFrame:
    metrics = get_cross_validation_metrics(model, X, y, cv=5)
    test_metrics = get_metrics(y_test, y_pred, y_prob)
    test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
    test = pd.DataFrame([test_metrics])
    test.set_index("fold", inplace=True)

    print_validated_model_accuracy(model, metrics)

    # Prepend test_metrics to metrics dataframe, export and display
    metrics = pd.concat([test, metrics])
    metrics.to_csv(f"../Data/model_metrics_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)
    return metrics

In [55]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metrics = run_cross_validation(X, y, y_test, y_pred, y_prod, False)
metrics

Model validation for LogisticRegression:
[0.9285714285714286, 0.9540816326530612, 0.9487179487179487, 0.9230769230769231, 0.9128205128205128]

Mean accuracy: 0.9335



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.954082,0.869565,0.769231,0.816327,0.981654,20,167,6,3
1,0.928571,0.652174,0.714286,0.681818,0.971350,15,167,6,8
2,0.954082,0.782609,0.818182,0.800000,0.968082,18,169,4,5
3,0.948718,0.782609,0.782609,0.782609,0.956522,18,167,5,5
4,0.923077,0.652174,0.681818,0.666667,0.935035,15,165,7,8
5,0.912821,0.521739,0.666667,0.585366,0.941860,12,166,6,11


In [56]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metric_smote = run_cross_validation(X_smote, y_smote, y_test_smote, y_pred_smote, y_prod_smote, True)
metric_smote

Model validation for LogisticRegression:
[0.9536231884057971, 0.9565217391304348, 0.9420289855072463, 0.9420289855072463, 0.9476744186046512]

Mean accuracy: 0.9484



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.959420,0.976744,0.943820,0.960000,0.979567,168,163,10,4
1,0.953623,0.976744,0.933333,0.954545,0.971502,168,161,12,4
2,0.956522,0.965116,0.948571,0.956772,0.976375,166,164,9,6
3,0.942029,0.930636,0.952663,0.941520,0.977047,161,164,8,12
4,0.942029,0.959538,0.927374,0.943182,0.960915,166,159,13,7
5,0.947674,0.953488,0.942529,0.947977,0.967922,164,162,10,8
